In [161]:
#general imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.model_selection import train_test_split


import lightgbm as lgb
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR



import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.model_selection import KFold

In [162]:
#Read dataset and make ydata-profiling dashboard

#from ydata_profiling import ProfileReport

df = pd.read_csv("../data/datasetRuntimeData.csv")
#Drop index col
df = df.drop(["Unnamed: 0"], axis=1)
#Remove special symbole
df['AutoML_adapter'] = df['AutoML_adapter'].str.replace(':', '')
#Apply one hot encoding
df = pd.get_dummies(df, columns=['AutoML_adapter'], prefix='', prefix_sep='')

#profile = ProfileReport(df, title="Profiling Report")
#profile.to_notebook_iframe()

In [163]:
df

,dataset_size_in_mb,dataset_rows,dataset_cols,missing_values,duplicated_rows,duplicated_cols,outliers,runtime_limit,autogluon,evalml,flaml,gama,h2o_automl,lama,pycaret,tpot
0,18.085659,539383,8,0,154622,0,99,40,False,False,False,False,True,False,False,False
1,18.085659,539383,8,0,154622,0,99,5,False,True,False,False,False,False,False,False
2,18.085659,539383,8,0,154622,0,99,640,False,False,False,True,False,False,False,False
3,18.085659,539383,8,0,154622,0,99,10,True,False,False,False,False,False,False,False
4,18.085659,539383,8,0,154622,0,99,20,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,0.197092,683,36,0,41,0,0,20,False,False,False,False,True,False,False,False
316,0.197092,683,36,0,41,0,0,5,True,False,False,False,False,False,False,False
317,0.197092,683,36,0,41,0,0,5,False,False,False,False,False,False,False,True
318,0.197092,683,36,0,41,0,0,20,False,False,True,False,False,False,False,False


In [164]:
#Split data all AutoML together

X = df.drop(["runtime_limit"], axis=1)
y = df["runtime_limit"]

from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X = pd.DataFrame(x_scaler.fit_transform(X), columns=X.columns)
y = pd.Series(y_scaler.fit_transform(y.values.reshape(-1, 1)).flatten())

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [165]:
# #Create prediction plot
# def create_prediction_plot(y_test, predictions):
#     prediction_results = pd.DataFrame({
#         'runtime_limit_is': y_test,
#         'runtime_limit_predicted': predictions
#     })

#     best_case_x = [0, 5, 10, 20, 40, 80, 160, 320, 640]
#     best_case_y = [0, 5, 10, 20, 40, 80, 160, 320, 640]
#     plt.figure(figsize=(10, 6))
#     sns.scatterplot(
#         x='runtime_limit_predicted', 
#         y='runtime_limit_is', 
#         data=prediction_results, 
#         color='gray', marker='o'  # Using a distinct color palette
#     )

#     plt.plot(best_case_x, best_case_y)
#     plt.xscale('log', base=10)  # Logarithmic scale for x-axis
#     plt.yscale('log', base=10)  # Logarithmic scale for y-axis


#     # Find the limits in log space
#     x_min, x_max = 1, 100
#     y_min, y_max = 1, np.exp(6.6)

#     # Determine the limits to make them symmetrical in log space
#     log_min = min(np.log10(x_min), np.log10(y_min))
#     log_max = max(np.log10(x_max), np.log10(y_max))

#     # Apply the symmetrical limits
#     plt.xlim([10**log_min, 10**log_max])
#     plt.ylim([10**log_min, 10**log_max])




#     plt.xlabel('Optimal runtime predicted')
#     plt.ylabel('Optimal runtime measured')
#     #plt.legend(title='AutoML Solution', bbox_to_anchor=(1.05, 0.5), loc='center left')
#     #plt.title('Actual vs Predicted Runtime Limits')
#     plt.grid(True)
#     #plt.legend(title='Series')
#     plt.show()
    

In [166]:


def train_model(model, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mae_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
        predictions = pd.Series(y_scaler.inverse_transform(predictions.reshape(-1, 1)).flatten())
        
        # Compute MAE
        mae = MAE(predictions, y_test_)
        mae_scores.append(mae)
        
    avg_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)  # Compute standard deviation
    
    print("###############################################")
    print("MAE Scores:", mae_scores)
    print(f"{type(model)} Average MAE across {n_splits} folds: {round(avg_mae, 4)} ± {round(std_mae, 4)}")
    
    
    return model

In [167]:
# def train_model(model,):
#     model.fit(X_train, y_train)
#     predictions = model.predict(X_test)
    
#     y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
#     predictions = pd.Series(y_scaler.inverse_transform(predictions.reshape(-1, 1)).flatten())

#     # Compute MAE
#     mae = MAE(predictions, y_test_)

#     print(f"{type(model)} Mean Absolute Error (MAE):", round(mae))
#     return model

In [168]:
def train_keras(n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mae_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Define the model
        model = Sequential([
            Dense(len(X_train.columns), activation='relu', input_shape=(len(X_train.columns),)),
            Dense(1, activation='linear')
        ])
        
        # Define ModelCheckpoint callback
        checkpoint_callback = ModelCheckpoint(
            './keras_models/2_1_1_best_model.keras', 
            monitor='val_mae',  # Monitor validation loss
            save_best_only=True,  # Save only when validation loss improves
            mode='auto'
        )
        
        early_stopping_callback = EarlyStopping(
            monitor='val_mae',  # Monitor validation MAE
            patience=25,  # Stop training if the validation MAE doesn't improve for 10 epochs
            restore_best_weights=True  # Restore the model weights from the best epoch
        )
        
        # Compile the model
        model.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError(), metrics=['mae'])
        model.fit(X_train, y_train, epochs=500, batch_size=16, verbose=0, callbacks=[checkpoint_callback, early_stopping_callback], validation_data=(X_test, y_test))
        
        model = tf.keras.models.load_model('./keras_models/2_1_1_best_model.keras')
        predictions = pd.Series(model.predict(X_test).flatten())
        
        y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
        predictions = pd.Series(y_scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten())
        
        # Compute MAE
        mae = MAE(predictions, y_test_)
        mae_scores.append(mae)
    
    avg_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)  # Compute standard deviation
    
    print("###############################################")
    print("MAE Scores:", mae_scores)
    print(f"{type(model)} Average MAE across {n_splits} folds: {round(avg_mae, 4)} ± {round(std_mae, 4)}")
    
    
    return model


In [169]:
# def train_keras():

#     # Define the model
#     model = Sequential([
#         Dense(len(X_train.columns), activation='relu', input_shape=(len(X_train.columns),)),  # Input layer (10 features)
#         #Dense(12, activation='relu'),  # Hidden layer 1
#         #Dense(6, activation='relu'),  # Hidden layer 2
#         Dense(1, activation='linear')  # Output layer (for binary classification)
#     ])

#     # Define ModelCheckpoint callback
#     checkpoint_callback = ModelCheckpoint(
#         './keras_models/2_1_1_best_model.h5', 
#         monitor='val_mae',  # Monitor validation loss
#         save_best_only=True,  # Save only when validation loss improves
#         mode='auto',  # We want to minimize validation loss
#         #verbose=1
#     )

#     # Compile the model
#     model.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError(), metrics=['mae'])
#     model.fit(X_train, y_train, epochs=500, batch_size=16, verbose=0, callbacks=[checkpoint_callback], validation_data=(X_test, y_test))
    
#     model = tf.keras.models.load_model('./keras_models/2_1_1_best_model.h5')
#     predictions = pd.Series(model.predict(X_test).flatten())


#     y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
#     predictions = pd.Series(y_scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten())


#     # Compute MAE
#     mae = MAE(predictions, y_test_)

#     print(f"{type(model)} Mean Absolute Error (MAE):", round(mae))
#     return model

In [170]:
def train_lgbm_model(n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    mae_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
        
        params = {
            'objective': 'regression',
            'metric': 'mae',
            'verbose': -1
        }
        
        num_round = 100
        model = lgb.train(params, train_data, num_round, valid_sets=[test_data])
        
        predictions = pd.Series(model.predict(X_test, num_iteration=model.best_iteration))
        
        y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
        predictions = pd.Series(y_scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten())
        
        mae = MAE(predictions, y_test_)
        mae_scores.append(mae)
    
    avg_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)  # Compute standard deviation
    
    print("###############################################")
    print("MAE Scores:", mae_scores)
    print(f"{type(model)} Average MAE across {n_splits} folds: {round(avg_mae, 4)} ± {round(std_mae, 4)}")
    
    
    return model


In [171]:

# def train_lgbm_model():
#     train_data = lgb.Dataset(X_train, label=y_train)
#     test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)



#     # Define parameters for the LightGBM model
#     params = {
#         'objective': 'regression',  # Set the objective as regression
#         'metric': 'mae',            # Use mean absolute error as the evaluation metric
#         'verbose': 1                # Disable verbose output
#     }

#     # Train the LightGBM model
#     num_round = 100
#     model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

#     # Make predictions on the test set
#     predictions = pd.Series(model.predict(X_test, num_iteration=model.best_iteration))
    

#     y_test_ = pd.Series(y_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten())
#     predictions = pd.Series(y_scaler.inverse_transform(predictions.values.reshape(-1, 1)).flatten())

#     # Compute MAE
#     mae = MAE(predictions, y_test)

#     print(f"{type(model)} Mean Absolute Error (MAE):", round(mae))
#     return model

In [172]:

models = { "Baseline": DummyRegressor(strategy="median"), 
          "LightGBM": None, 
          "Linear Regression": LinearRegression(), 
          "Decision Tree": DecisionTreeRegressor(random_state=42), 
          "Sklearn Neural Network": MLPRegressor(random_state=42), 
          "Keras Neural Network": None, 
          "Ridge": Ridge(), 
          "Lasso": Lasso(), 
          "Elastic": ElasticNet(), 
          "Random Forest": RandomForestRegressor(), 
          "Bayesian": BayesianRidge(), 
          "SVM": SVR()}


for model_name, model in models.items():
    if model_name == "LightGBM":
        models[model_name] = train_lgbm_model()
    elif model_name == "Keras Neural Network":
        models[model_name] = train_keras()
    else:
        models[model_name] = train_model(model)


###############################################
MAE Scores: [104.375, 105.15625, 155.15625, 121.5625, 106.40625, 93.4375, 51.71875, 85.3125, 75.46875, 109.84375]
<class 'sklearn.dummy.DummyRegressor'> Average MAE across 10 folds: 100.8438 ± 26.2209
###############################################
MAE Scores: [113.55323040477292, 124.84343273474835, 159.3062971691807, 154.26554574392338, 130.18922868181738, 107.90234932159957, 86.85027768744823, 104.56247642941943, 130.9793230422262, 130.9543923130465]
<class 'lightgbm.basic.Booster'> Average MAE across 10 folds: 124.3407 ± 21.0325
###############################################
MAE Scores: [107.059326171875, 106.21536254882812, 150.897216796875, 152.530517578125, 111.43310546875, 108.013916015625, 92.574462890625, 106.3970947265625, 126.4208984375, 131.57181838189484]
<class 'sklearn.linear_model._base.LinearRegression'> Average MAE across 10 folds: 119.3114 ± 19.2224
###############################################
MAE Scores: [136.25, 

c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


c:\Users\alex\Desktop\MetaAutoML\utils\analysis\ai_based_optimization\predicting_optimal_runtime\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
###############################################
MAE Scores: [90.77830832451582, 85.03746999800205, 132.4339361190796, 138.99142545461655, 105.61806264519691, 92.79855471104383, 58.75854334235191, 76.4926639534533, 78.53938563168049, 113.01963271200657]
<class 'keras.src.models.sequential.Sequential'> Average MAE across 10 folds: 97.2468 ± 24.0257
###############################################
MAE Scores: [104.00850144534182, 107.31159310731972, 153.28851155256277, 152.8193703695885, 114.4733857189099, 102.41316872398124, 92.43612812348644, 102.94385399530148, 120.37930239549138, 132.81458073198908]
<class 'sklearn.linear_model._ridge.Ridge'> Average MAE across 10 folds: 118.2888 ± 20.3164
###############################################
MAE Scores: [118.50260416666667, 144.55729166666669, 182.861328125, 160.78125, 130.59895833333334, 124.61805555555556, 99.990234375, 112.53472222222223, 125.771484375, 132.48046875]
<class 'sklearn.linear_model._coo